![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.Pretrained_Clinical_Pipelines.ipynb)

# 11. Pretrained_Clinical_Pipelines

## Colab Setup

In [1]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

license_keys.keys()

Saving license_keys_272.json to license_keys_272.json


dict_keys(['PUBLIC_VERSION', 'JSL_VERSION', 'SECRET', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'SPARK_OCR_LICENSE', 'SPARK_OCR_SECRET'])

In [2]:
license_keys['JSL_VERSION']

'2.7.2'

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys['SECRET']

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
version = license_keys['PUBLIC_VERSION']
jsl_version = license_keys['JSL_VERSION']

! pip install --ignore-installed -q pyspark==2.4.4

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

! pip install --ignore-installed -q spark-nlp==$version

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

from sparknlp.pretrained import PretrainedPipeline

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}
spark = sparknlp_jsl.start(secret, params=params)


<b>  if you want to work with Spark 2.3 </b>
```
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-2.3.0/spark-2.3.0-bin-hadoop2.7.tgz

!tar xf spark-2.3.0-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-2.3.0-bin-hadoop2.7"
! java -version

import findspark
findspark.init()
from pyspark.sql import SparkSession

! pip install --ignore-installed -q spark-nlp==2.5.5
import sparknlp

spark = sparknlp.start(spark23=True)
```

## Pretrained Pipelines

In order to save you from creating a pipeline from scratch, Spark NLP also has a pre-trained pipelines that are already fitted using certain annotators and transformers according to various use cases.

Here is the list of clinical pre-trained pipelines: 

> These clinical pipelines are trained with `embeddings_healthcare_100d` and accuracies might be 1-2% lower than `embeddings_clinical` which is 200d.

**1.   explain_clinical_doc_carp** :

> a pipeline with `ner_clinical`, `assertion_dl`, `re_clinical` and `ner_posology`. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

**2.   explain_clinical_doc_era** :

> a pipeline with `ner_clinical_events`, `assertion_dl` and `re_temporal_events_clinical`. It will extract clinical entities, assign assertion status and find temporal relationships between clinical entities.

**3.   recognize_entities_posology** :

> a pipeline with `ner_posology`. It will only extract medication entities.


** Since 3rd pipeline is already a subset of 1st and 2nd pipeline, we will only cover the first two pipelines in this notebook.

**4.   explain_clinical_doc_ade** :

> a pipeline for `Adverse Drug Events (ADE)` with `ner_ade_biobert`, `assertiondl_biobert` and `classifierdl_ade_conversational_biobert`. It will extract `ADE` and `DRUG` clinical entities, assign assertion status to `ADE` entities, and then assign ADE status to a text (`True` means ADE, `False` means not related to ADE).

**letter codes in the naming conventions:**

> c : ner_clinical

> e : ner_clinical_events

> r : relation extraction

> p : ner_posology

> a : assertion

> ade : adverse drug events

**Relation Extraction types:**

`re_clinical` >> TrIP (improved), TrWP (worsened), TrCP (caused problem), TrAP (administered), TrNAP (avoided), TeRP (revealed problem), TeCP (investigate problem), PIP (problems related)

`re_temporal_events_clinical` >> `AFTER`, `BEFORE`, `OVERLAP`


## 1.explain_clinical_doc_carp 

a pipeline with ner_clinical, assertion_dl, re_clinical and ner_posology. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

In [4]:
pipeline = PretrainedPipeline('explain_clinical_doc_carp', 'en', 'clinical/models')

explain_clinical_doc_carp download started this may take some time.
Approx size to download 526.5 MB
[OK!]


In [5]:
pipeline.model.stages

[DocumentAssembler_8aeb50463a0d,
 SentenceDetector_635a56ed49ab,
 REGEX_TOKENIZER_6f0bd3b85024,
 WORD_EMBEDDINGS_MODEL_a5c1afb0b657,
 POS_be8d41751649,
 NerDLModel_706522935b2e,
 NerConverter_b818c367ba56,
 dependency_68159e3d6dac,
 NerDLModel_01b90ff03d9e,
 NerConverter_335d7d4208fc,
 RelationExtractionModel_0a71121bf321,
 ASSERTION_DL_941a00a50db4]

In [ ]:
# Load pretrained pipeline from local disk:

# >> pipeline_local = PretrainedPipeline.from_disk('/root/cache_pretrained/explain_clinical_doc_carp_en_2.5.5_2.4_1597841630062')

In [6]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus, used to take metformin 1000 mg two times a day, presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .
She was seen by the endocrinology service and discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals.
"""

annotations = pipeline.annotate(text)

annotations.keys()


dict_keys(['sentences', 'clinical_ner_tags', 'document', 'ner_chunks', 'clinical_ner_chunks', 'ner_tags', 'assertion', 'clinical_relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [7]:
import pandas as pd

rows = list(zip(annotations['tokens'], annotations['clinical_ner_tags'], annotations['ner_tags'], annotations['pos_tags'], annotations['dependencies']))

df = pd.DataFrame(rows, columns = ['tokens','clinical_ner_tags','posology_ner_tags','POS_tags','dependencies'])

df.head(20)

,tokens,clinical_ner_tags,posology_ner_tags,POS_tags,dependencies
0,A,O,O,DD,female
1,28-year-old,O,O,NN,female
2,female,O,O,NN,ROOT
3,with,O,O,II,history
4,a,O,O,DD,history
5,history,O,O,NN,female
6,of,O,O,II,history
7,gestational,B-PROBLEM,O,JJ,of
8,diabetes,I-PROBLEM,O,NN,mellitus
9,mellitus,I-PROBLEM,O,NN,gestational


In [8]:
text = 'Patient has a headache for the last 2 weeks and appears anxious when she walks fast. No alopecia noted. She denies pain'

result = pipeline.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(result['clinical_ner_chunks'],result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,a headache,PROBLEM,present
1,anxious,PROBLEM,present
2,alopecia,PROBLEM,absent
3,pain,PROBLEM,absent


In [9]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 
12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

result = pipeline.fullAnnotate(text)[0]

chunks=[]
entities=[]
begins=[]
ends=[]

for n in result['ner_chunks']:
    
    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity']) 
        
df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,1 unit,28,33,Dosage
1,Advil,38,42,Drug
2,for 5 days,44,53,Duration
3,1 unit,96,101,Dosage
4,Metformin,106,114,Drug
5,daily,116,120,Frequency
6,40 units,190,197,Dosage
7,insulin glargine,202,217,Drug
8,at night,219,226,Frequency
9,12 units,231,238,Dosage


In [10]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  rel_df.confidence = rel_df.confidence.astype(float)
  
  return rel_df


In [11]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus, used to take metformin 1000 mg two times a day, presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .
She was seen by the endocrinology service and discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals.
"""

annotations = pipeline.fullAnnotate(text)

rel_df = get_relations_df (annotations, 'clinical_relations')

rel_df

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,TrAP,PROBLEM,39,67,gestational diabetes mellitus,TREATMENT,83,91,metformin,0.999896
1,TrCP,PROBLEM,155,162,polyuria,PROBLEM,166,175,polydipsia,0.647005
2,TrCP,PROBLEM,155,162,polyuria,PROBLEM,179,191,poor appetite,0.909743
3,TrWP,PROBLEM,155,162,polyuria,PROBLEM,199,206,vomiting,0.524460
4,TrCP,PROBLEM,166,175,polydipsia,PROBLEM,179,191,poor appetite,0.644446
5,TrAP,PROBLEM,166,175,polydipsia,PROBLEM,199,206,vomiting,0.503557
6,TrAP,PROBLEM,179,191,poor appetite,PROBLEM,199,206,vomiting,0.993527


In [12]:
text ="""
he patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . 
However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
"""

annotations = pipeline.fullAnnotate(text)

rel_df = get_relations_df (annotations, 'clinical_relations')

rel_df[rel_df.confidence>0.9]


,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
2,TrAP,PROBLEM,249,262,still elevated,TEST,272,288,serum bicarbonate,0.997434
3,TrAP,PROBLEM,249,262,still elevated,TEST,352,357,lipase,0.996670
4,TrAP,PROBLEM,249,262,still elevated,TEST,366,368,U/L,0.939070
6,TrIP,TEST,272,288,serum bicarbonate,TEST,366,368,U/L,0.926109
8,TrAP,TEST,352,357,lipase,TEST,366,368,U/L,0.999179
10,TrAP,PROBLEM,429,436,elevated,TREATMENT,495,515,the chylomicron layer,0.999306
12,TrAP,TREATMENT,633,647,an insulin drip,TEST,663,665,HTG,0.999923
13,TrAP,TEST,663,665,HTG,PROBLEM,672,699,a reduction in the anion gap,0.998082
14,TrAP,TEST,663,665,HTG,TEST,711,723,triglycerides,0.955180
15,TrAP,PROBLEM,672,699,a reduction in the anion gap,TEST,711,723,triglycerides,0.947221


## **2.   explain_clinical_doc_era** 

> a pipeline with `ner_clinical_events`, `assertion_dl` and `re_temporal_events_clinical`. It will extract clinical entities, assign assertion status and find temporal relationships between clinical entities.



In [13]:
era_pipeline = PretrainedPipeline('explain_clinical_doc_era', 'en', 'clinical/models')

explain_clinical_doc_era download started this may take some time.
Approx size to download 512.8 MB
[OK!]


In [14]:
era_pipeline.model.stages

[DocumentAssembler_f548f799ea2a,
 SentenceDetector_249f783c340c,
 REGEX_TOKENIZER_209534c638ca,
 POS_be8d41751649,
 dependency_68159e3d6dac,
 WORD_EMBEDDINGS_MODEL_a5c1afb0b657,
 NerDLModel_3c27190d1858,
 NerConverter_9fff8fa39dbd,
 RelationExtractionModel_fb3be959a99e,
 NerConverter_50212b459b4b,
 ASSERTION_DL_941a00a50db4]

In [ ]:
text ="""She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.
She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 
12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge. 
"""

result = era_pipeline.fullAnnotate(text)[0]


In [15]:
result.keys()

dict_keys(['sentences', 'clinical_ner_tags', 'document', 'ner_chunks', 'clinical_ner_chunks', 'ner_tags', 'assertion', 'clinical_relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [16]:
import pandas as pd

chunks=[]
entities=[]
begins=[]
ends=[]

for n in result['clinical_ner_chunks']:
    
    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity']) 
        
df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,Metformin,106,114,TREATMENT
1,insulin glargine,202,217,TREATMENT
2,insulin lispro,243,256,TREATMENT
3,metformin,275,283,TREATMENT


In [18]:

chunks=[]
entities=[]
status=[]

for n,m in zip(result['clinical_ner_chunks'],result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

,chunks,entities,assertion
0,Metformin,TREATMENT,present
1,insulin glargine,TREATMENT,present
2,insulin lispro,TREATMENT,present
3,metformin,TREATMENT,present


In [19]:
annotations = era_pipeline.fullAnnotate(text)

rel_df = get_relations_df (annotations, 'clinical_relations')

rel_df


,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,BEFORE,OCCURRENCE,26,33,admitted,PROBLEM,39,56,starvation ketosis,0.999999
1,BEFORE,EVIDENTIAL,67,74,reported,PROBLEM,76,91,poor oral intake,1.000000
2,OVERLAP,EVIDENTIAL,67,74,reported,DURATION,97,106,three days,0.999968
3,OVERLAP,PROBLEM,76,91,poor oral intake,DURATION,97,106,three days,1.000000
4,OVERLAP,PROBLEM,76,91,poor oral intake,OCCURRENCE,117,125,admission,1.000000
5,BEFORE,DURATION,97,106,three days,OCCURRENCE,117,125,admission,1.000000
6,AFTER,TEST,140,154,serum chemistry,EVIDENTIAL,181,192,presentation,1.000000
7,OVERLAP,TEST,140,154,serum chemistry,EVIDENTIAL,194,201,revealed,1.000000
8,AFTER,DURATION,165,173,six hours,EVIDENTIAL,181,192,presentation,1.000000
9,AFTER,DURATION,165,173,six hours,EVIDENTIAL,194,201,revealed,0.561261


In [ ]:
annotations[0]['clinical_relations']

## 3.explain_clinical_doc_ade 

a pipeline for `Adverse Drug Events (ADE)` with `ner_ade_healthcare`, and `classifierdl_ade_biobert`. It will extract `ADE` and `DRUG` clinical entities, and then assign ADE status to a text(`Negative` means ADE, `Neutral` means not related to ADE).

In [20]:
ade_pipeline = PretrainedPipeline('explain_clinical_doc_ade', 'en', 'clinical/models')

explain_clinical_doc_ade download started this may take some time.
Approx size to download 426.7 MB
[OK!]


In [21]:
ade_pipeline.fullAnnotate("I feel a bit drowsy & have a little blurred vision, so far no gastric problems.")[0]['class'][0].metadata

{'sentence': '0', 'False': '3.8420252E-4', 'True': '0.9996158'}

In [22]:
text = "I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50."

import pandas as pd

chunks = []
entities = []
begin =[]
end = []

print ('sentence:', text)
print()

result = ade_pipeline.fullAnnotate(text)

print ('ADE status:', result[0]['class'][0].result)

print ('prediction probability>> True : ', result[0]['class'][0].metadata['True'], \
        'False: ', result[0]['class'][0].metadata['False'])

for n in result[0]['ner_chunk']:

  begin.append(n.begin)
  end.append(n.end)
  chunks.append(n.result)
  entities.append(n.metadata['entity']) 

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                'begin': begin, 'end': end})

df


sentence: I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.

ADE status: True
prediction probability>> True :  0.99847335 False:  0.0015265922


,chunks,entities,begin,end
0,bit drowsy,ADE,9,18
1,little blurred vision,ADE,29,49
2,gastric problems,ADE,62,77
3,Arthrotec,DRUG,95,103
4,feel a bit weird,ADE,438,453


#### with AssertionDL

In [23]:
import pandas as pd

text = 'I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.'

print (text)

light_result = ade_pipeline.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ass_ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.


,chunks,entities,assertion
0,stroke,ADE,possible
1,blood clots that traveled to my eye,ADE,present
